## Facebook AI Similarity Search

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader('speech.txt')
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=200,chunk_overlap=30)
docs=text_splitter.split_documents(documents)
docs

Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200


[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [3]:
embeddings=OllamaEmbeddings(model="gemma:2b")
db=FAISS.from_documents(docs,embeddings)
db

In [6]:
query="What does the speaker believe is the main reason the United States should enter yhe war?"
docs=db.similarity_search(query)
docs[0].page_content

'I have said nothing of the governments allied with the Imperial government of Germany because they have not made war upon us or challenged us to defend our right and our honor. The Austro-Hungarian government has, indeed, avowed its unqualified endorsement and acceptance of the reckless and lawless submarine warfare adopted now without disguise by the Imperial German government, and it has therefore not been possible for this government to receive Count Tarnowski, the ambassador recently accredited to this government by the Imperial and Royal government of Austria-Hungary; but that government has not actually engaged in warfare against citizens of the United States on the seas, and I take the liberty, for the present at least, of postponing a discussion of our relations with the authorities at Vienna. We enter this war only where we are clearly forced into it because there are no other means of defending our rights.'

In [8]:
## Retriever
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'I have said nothing of the governments allied with the Imperial government of Germany because they have not made war upon us or challenged us to defend our right and our honor. The Austro-Hungarian government has, indeed, avowed its unqualified endorsement and acceptance of the reckless and lawless submarine warfare adopted now without disguise by the Imperial German government, and it has therefore not been possible for this government to receive Count Tarnowski, the ambassador recently accredited to this government by the Imperial and Royal government of Austria-Hungary; but that government has not actually engaged in warfare against citizens of the United States on the seas, and I take the liberty, for the present at least, of postponing a discussion of our relations with the authorities at Vienna. We enter this war only where we are clearly forced into it because there are no other means of defending our rights.'

In [10]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='3e1be5dc-7fa8-4c27-9a0b-842b71b78a15', metadata={'source': 'speech.txt'}, page_content='I have said nothing of the governments allied with the Imperial government of Germany because they have not made war upon us or challenged us to defend our right and our honor. The Austro-Hungarian government has, indeed, avowed its unqualified endorsement and acceptance of the reckless and lawless submarine warfare adopted now without disguise by the Imperial German government, and it has therefore not been possible for this government to receive Count Tarnowski, the ambassador recently accredited to this government by the Imperial and Royal government of Austria-Hungary; but that government has not actually engaged in warfare against citizens of the United States on the seas, and I take the liberty, for the present at least, of postponing a discussion of our relations with the authorities at Vienna. We enter this war only where we are clearly forced into it because there are no othe

In [12]:
embeddings_vector=embeddings.embed_query(query)
embeddings_vector

[-0.009366786107420921,
 1.1646645069122314,
 0.21177196502685547,
 1.0673235654830933,
 -0.07919726520776749,
 0.3662238121032715,
 0.6750373840332031,
 0.3257877230644226,
 0.5738396048545837,
 0.3322265148162842,
 -0.8843563795089722,
 -0.059876881539821625,
 -0.7795304656028748,
 1.0170338153839111,
 1.231947898864746,
 -0.37646397948265076,
 2.3236494064331055,
 0.7650890946388245,
 1.1957381963729858,
 0.7081719040870667,
 0.6689799427986145,
 -0.707839846611023,
 1.382173776626587,
 0.4191129803657532,
 -0.9031259417533875,
 -0.9875002503395081,
 -1.631244421005249,
 -1.6881688833236694,
 -0.969299852848053,
 -1.396291971206665,
 2.7636165618896484,
 -0.5457465648651123,
 1.401669979095459,
 -0.26699236035346985,
 -1.0769439935684204,
 -0.3615845739841461,
 0.19534146785736084,
 -1.1234208345413208,
 0.07667117565870285,
 -0.6305902004241943,
 -0.36777105927467346,
 -0.29696372151374817,
 0.5811645984649658,
 -0.5948824286460876,
 -0.7650889754295349,
 0.708993673324585,
 0.6679

In [15]:
docs_and_score=db.similarity_search_by_vector(embeddings_vector)
docs_and_score

[Document(id='3e1be5dc-7fa8-4c27-9a0b-842b71b78a15', metadata={'source': 'speech.txt'}, page_content='I have said nothing of the governments allied with the Imperial government of Germany because they have not made war upon us or challenged us to defend our right and our honor. The Austro-Hungarian government has, indeed, avowed its unqualified endorsement and acceptance of the reckless and lawless submarine warfare adopted now without disguise by the Imperial German government, and it has therefore not been possible for this government to receive Count Tarnowski, the ambassador recently accredited to this government by the Imperial and Royal government of Austria-Hungary; but that government has not actually engaged in warfare against citizens of the United States on the seas, and I take the liberty, for the present at least, of postponing a discussion of our relations with the authorities at Vienna. We enter this war only where we are clearly forced into it because there are no other

In [16]:
## Saving and Loadin
db.save_local('faiss_index')

In [19]:
new_df=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)

In [21]:
new_df.similarity_search(query)

[Document(id='3e1be5dc-7fa8-4c27-9a0b-842b71b78a15', metadata={'source': 'speech.txt'}, page_content='I have said nothing of the governments allied with the Imperial government of Germany because they have not made war upon us or challenged us to defend our right and our honor. The Austro-Hungarian government has, indeed, avowed its unqualified endorsement and acceptance of the reckless and lawless submarine warfare adopted now without disguise by the Imperial German government, and it has therefore not been possible for this government to receive Count Tarnowski, the ambassador recently accredited to this government by the Imperial and Royal government of Austria-Hungary; but that government has not actually engaged in warfare against citizens of the United States on the seas, and I take the liberty, for the present at least, of postponing a discussion of our relations with the authorities at Vienna. We enter this war only where we are clearly forced into it because there are no other